In [1]:
import os, subprocess
import yaml
import json
from tensorflow.python.tools import saved_model_utils

--- 

#### CONFIG

In [2]:
PROJECT='surface-water-278805'
OUTPUT_BAND='water'
CONFIG={
    'autoScaling': { 'minNodes': 10 }
}

In [3]:
TRAIN_NAME='xception-xception_seg_long.wcce-fromlogits-lsmooth0p1_20200707_052215'
RUN='sw_r7'
MODEL_NAME='xception_seg_long'
VERSION=1
IS_NEW_MODEL=False
DRY_RUN=False

---

#### CONSTANTS

In [4]:
EEIFY_CMD=(
    "earthengine model prepare --source_dir {} --dest_dir {} "
    "--input \'{}\' --output \'{}\'" )

CREATE_MODEL_CMD='gcloud ai-platform models create {} --project {}'

CREATE_VERSION_CMD=(
    'gcloud ai-platform versions create {} '
        '--project {} '
        '--model {} '
        '--origin {} '
        '--framework "TENSORFLOW" '
        '--runtime-version 2.1 '
        '--python-version 3.7 '
        '--config={}' )

In [5]:
TMP_YAML_FILE='_config.tmp.yaml'

---

#### METHODS

In [6]:
def _write_yaml(destination_yaml_file,**kwargs):
    with open(destination_yaml_file, 'w') as file:
        documents = yaml.dump(kwargs, file)

        
def _process_cmd(cmd,description='run',dry_run=False):
    print(f'{description}:',cmd)
    if not dry_run:
        result=subprocess.run([cmd], stdout=subprocess.PIPE, shell=True)
        print(result.stdout.decode('utf-8'))

In [7]:
def get_dirs(model_name,run_folder,version=1,is_dev=False):
    src=f'gs://sw-ai-platform/{run_folder}/output/{model_name}/model'
    if isinstance(version,int):
        version=f'v{version}'
    dest='gs://gee-models/models'
    if is_dev:
        dest=f'dest/{dev}'
    dest=f'{dest}/{model_name}/{version}'
    return src, dest, version

In [8]:
def eeify(src_dir,dest_dir,output_band=OUTPUT_BAND,dry_run=DRY_RUN):
    meta_graph_def = saved_model_utils.get_meta_graph_def(src_dir, 'serve')
    inputs=meta_graph_def.signature_def['serving_default'].inputs
    outputs=meta_graph_def.signature_def['serving_default'].outputs
    input_dict={ next(iter(inputs.values())).name: "array" }
    output_dict={ next(iter(outputs.values())).name: output_band }
    print('-'*100)
    cmd=EEIFY_CMD.format(src_dir,dest_dir,json.dumps(input_dict),json.dumps(output_dict))
    _process_cmd(cmd,description='eeifying model',dry_run=dry_run)
    print('-'*100)
    

def create_model(model_name,project=PROJECT,dry_run=DRY_RUN):
    cmd=CREATE_MODEL_CMD.format(model_name,project)
    _process_cmd(cmd,description='create model',dry_run=dry_run)
        
        
def create_version(model_name,model_dir,version,config=CONFIG,project=PROJECT,dry_run=DRY_RUN):
    _write_yaml(TMP_YAML_FILE,**config)
    cmd=CREATE_VERSION_CMD.format(version,project,model_name,model_dir,TMP_YAML_FILE)
    _process_cmd(cmd,description='create version',dry_run=dry_run)
    os.remove(TMP_YAML_FILE)

---

#### RUN

In [9]:
src_dir, ee_dir, version=get_dirs(TRAIN_NAME,RUN,VERSION)
print('-'*100)
print('src:',src_dir)
print('geeified:',ee_dir)
print('model:',MODEL_NAME)
print('model_version:',version)
print('-'*100)

----------------------------------------------------------------------------------------------------
src: gs://sw-ai-platform/sw_r7/output/xception-xception_seg_long.wcce-fromlogits-lsmooth0p1_20200707_052215/model
geeified: gs://gee-models/models/xception-xception_seg_long.wcce-fromlogits-lsmooth0p1_20200707_052215/v1
model: xception_seg_long
model_version: v1
----------------------------------------------------------------------------------------------------


In [10]:
eeify(src_dir,ee_dir)

----------------------------------------------------------------------------------------------------
eeifying model: earthengine model prepare --source_dir gs://sw-ai-platform/sw_r7/output/xception-xception_seg_long.wcce-fromlogits-lsmooth0p1_20200707_052215/model --dest_dir gs://gee-models/models/xception-xception_seg_long.wcce-fromlogits-lsmooth0p1_20200707_052215/v1 --input '{"serving_default_input:0": "array"}' --output '{"StatefulPartitionedCall:0": "water"}'
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

Success: model at 'gs://gee-models/models/xception-xception_seg_long.wcce-fromlogits-lsmooth0p1_20200707_052215/v1' is ready to be hosted in AI Platform.

----------------------------------------------------------------------------------------------------


In [11]:
if IS_NEW_MODEL:
    create_model(MODEL_NAME)

In [12]:
create_version(MODEL_NAME,ee_dir,version)

create version: gcloud ai-platform versions create v1 --project surface-water-278805 --model xception_seg_long --origin gs://gee-models/models/xception-xception_seg_long.wcce-fromlogits-lsmooth0p1_20200707_052215/v1 --framework "TENSORFLOW" --runtime-version 2.1 --python-version 3.7 --config=_config.tmp.yaml

